In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'prod'

In [ ]:
import src.config
from deltalake import DeltaTable
from pathlib import Path
import pandas as pd


In [ ]:
TABLE_PATH = src.config.SILVER / "pers_dim"
df = DeltaTable(TABLE_PATH).to_pandas()
df['eastern_time'] = df['load_timestamp_utc'].dt.tz_convert('America/New_York')

In [ ]:
pers_dim = df.copy()
pers_dim

In [ ]:
TABLE_PATH = src.config.SILVER / "customer_address_link"
df = DeltaTable(TABLE_PATH).to_pandas()
df['eastern_time'] = df['load_timestamp_utc'].dt.tz_convert('America/New_York')

In [ ]:
customer_address_link = df.copy()
customer_address_link

In [ ]:
TABLE_PATH = src.config.SILVER / "address"
df = DeltaTable(TABLE_PATH).to_pandas()
df['eastern_time'] = df['load_timestamp_utc'].dt.tz_convert('America/New_York')

In [ ]:
address = df.copy()
address

In [ ]:
customer_address_link = customer_address_link[customer_address_link['addrusecd'] == 'PRI'].copy()

In [ ]:
import cdutils.input_cleansing

In [ ]:
customer_address_link_schema = {
    'addrnbr':'str'
}

customer_address_link = cdutils.input_cleansing.cast_columns(customer_address_link, customer_address_link_schema)

In [ ]:
pers_dim = pers_dim.merge(customer_address_link, on='customer_id', how='left').merge(address, on='addrnbr', how='left')

In [ ]:
pers_dim